In [46]:
import pandas as pd 

df=pd.read_csv("pe_class2.csv")
city=list(df["Cities"].str.split(","))

# to capture date and month
dept_date=[]
dept_month=[]
date=[x[1] for x in city]
for i in date:
    k=i.strip()
    dept_date.append(k.split(" ")[0])
    dept_month.append(k.split(" ")[1])
dept_city=[]
arrival_city=[]

# to capture departure city and arrival city
c=[x[0] for x in city]
for i in c:
    dept_city.append(i.split(" to ")[0])
    arrival_city.append(i.split(" to ")[1])

# to capture the number of stops 
stops=[]
for i in df["layeover"]:
    if i[0]=="N":
        stops.append(0)
    else:
        stops.append(i[0])

        
# to capture the layover cities
# layover_cities=[]
# for i in df["layeover"]:
#     if "via" in i:
#         layover_cities.append(i.split(" via ")[1])
#     else:
#         layover_cities.append("No Stops")
        
#creating dataframe
df["Dept_city"]=dept_city
df["Dept_date"]=dept_date
df["Dept_month"]=dept_month
df["arrival_city"]=arrival_city
df["stops"]=stops
#df["layover_cities"]=layover_cities


# droping the usless columns
#df.drop(columns=["Cities","layeover"],axis=1,inplace=True)

In [47]:
df.head()

,Price,Cities,departure_time,arrival_time,layeover,Airline,Scraping_Time,Cabin,Dept_city,Dept_date,Dept_month,arrival_city,stops
0,"6,855","New Delhi to Chennai , 10 May",18:55,21:35,Non stop,Spicejet,2021-05-10 16:34:04,PE,New Delhi,10,May,Chennai,0
1,"12,168","New Delhi to Chennai , 10 May",19:10,13:15,1 stop via Patna,Spicejet,2021-05-10 16:34:04,PE,New Delhi,10,May,Chennai,1
2,"10,096","New Delhi to Guwahati , 10 May",19:10,20:25,1 stop via Patna,Spicejet,2021-05-10 16:36:14,PE,New Delhi,10,May,Guwahati,1
3,"11,788","New Delhi to Patna , 10 May",19:10,20:50,Non stop,Spicejet,2021-05-10 16:37:14,PE,New Delhi,10,May,Patna,0
4,"6,147","Mumbai to New Delhi , 10 May",19:45,21:55,Non stop,Vistara,2021-05-10 16:44:39,PE,Mumbai,10,May,New Delhi,0


In [48]:
df.shape

(13139, 13)

In [49]:
df.drop_duplicates(inplace=True)

In [50]:
# more columns can be added such as duration,year,weekday,dayofweek ,class of the seat

In [51]:
#Creating Departure Date Columns

dates1=[]
for i in range(df.shape[0]):
    x="2021-05-"+str(df["Dept_date"].iloc[i])+" "+str(df["departure_time"].iloc[i])
    dates1.append(x)
df["Departure_Date"]=pd.to_datetime(dates1)

In [52]:
#Creating Arrival Date Columns
dates1=[]
for i in range(df.shape[0]):
    x="2021-05-"+str(df["Dept_date"].iloc[i])+" "+str(df["arrival_time"].iloc[i])
    dates1.append(x)
df["Arrival_Date"]=pd.to_datetime(dates1)

In [53]:
#Creating Duration Column
import numpy as np

df["duration_time"]=df["Arrival_Date"]-df["Departure_Date"]
df["duration"]=df["duration_time"]/ np.timedelta64(1, 'm')
for i in range(df.shape[0]):
    if df["duration"].iloc[i]<0:
        df["duration"].iloc[i]=1440+df["duration"].iloc[i]

C:\Users\Dell\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [54]:
#Creating columns like Weekday/Weekname
df["weekday"]=pd.to_datetime(df["Departure_Date"]).dt.weekday
df["day_name"]=pd.to_datetime(df["Departure_Date"]).dt.day_name()

In [55]:
#Creating columns like wehther its a morning,evening,night or afternoon flight from departure time & arrival time of the flight
df["dept_hours"]=pd.to_datetime(df["Departure_Date"]).dt.hour
df["arrival_hours"]=pd.to_datetime(df["Arrival_Date"]).dt.hour

def depart(x):
    d=[]
    for i in range(df.shape[0]):
        if x.iloc[i]>=16 and x.iloc[i]<21:
            d.append("Evening_flight")
    
        if x.iloc[i]>=21 or x.iloc[i]<5:
            d.append("Night_flight")
    
        if x.iloc[i]>=5 and x.iloc[i]<11:
            d.append("Morning_flight")
    
        if x.iloc[i]>=11 and x.iloc[i]<16:
            d.append("Afternoon_flight")
    return d

df["Dept_flights_time"]=depart(df["dept_hours"])
df["arrival_flights_time"]=depart(df["arrival_hours"])


In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13139 entries, 0 to 13138
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype          
---  ------                --------------  -----          
 0   Price                 13139 non-null  object         
 1   Cities                13139 non-null  object         
 2   departure_time        13139 non-null  object         
 3   arrival_time          13139 non-null  object         
 4   layeover              13139 non-null  object         
 5   Airline               13139 non-null  object         
 6   Scraping_Time         13139 non-null  object         
 7   Cabin                 13139 non-null  object         
 8   Dept_city             13139 non-null  object         
 9   Dept_date             13139 non-null  object         
 10  Dept_month            13139 non-null  object         
 11  arrival_city          13139 non-null  object         
 12  stops                 13139 non-null  object         
 13  D

In [57]:
df["weekday"].value_counts()

4    2258
3    2237
5    2061
6    1862
1    1655
2    1621
0    1445
Name: weekday, dtype: int64

In [58]:
df["Price"]=df["Price"].str.replace(",","")
df["Price"]=df["Price"].astype("int")

In [59]:
df.head()

,Price,Cities,departure_time,arrival_time,layeover,Airline,Scraping_Time,Cabin,Dept_city,Dept_date,...,Departure_Date,Arrival_Date,duration_time,duration,weekday,day_name,dept_hours,arrival_hours,Dept_flights_time,arrival_flights_time
0,6855,"New Delhi to Chennai , 10 May",18:55,21:35,Non stop,Spicejet,2021-05-10 16:34:04,PE,New Delhi,10,...,2021-05-10 18:55:00,2021-05-10 21:35:00,0 days 02:40:00,160.0,0,Monday,18,21,Evening_flight,Night_flight
1,12168,"New Delhi to Chennai , 10 May",19:10,13:15,1 stop via Patna,Spicejet,2021-05-10 16:34:04,PE,New Delhi,10,...,2021-05-10 19:10:00,2021-05-10 13:15:00,-1 days +18:05:00,1085.0,0,Monday,19,13,Evening_flight,Afternoon_flight
2,10096,"New Delhi to Guwahati , 10 May",19:10,20:25,1 stop via Patna,Spicejet,2021-05-10 16:36:14,PE,New Delhi,10,...,2021-05-10 19:10:00,2021-05-10 20:25:00,0 days 01:15:00,75.0,0,Monday,19,20,Evening_flight,Evening_flight
3,11788,"New Delhi to Patna , 10 May",19:10,20:50,Non stop,Spicejet,2021-05-10 16:37:14,PE,New Delhi,10,...,2021-05-10 19:10:00,2021-05-10 20:50:00,0 days 01:40:00,100.0,0,Monday,19,20,Evening_flight,Evening_flight
4,6147,"Mumbai to New Delhi , 10 May",19:45,21:55,Non stop,Vistara,2021-05-10 16:44:39,PE,Mumbai,10,...,2021-05-10 19:45:00,2021-05-10 21:55:00,0 days 02:10:00,130.0,0,Monday,19,21,Evening_flight,Night_flight


In [60]:
df["stops"]=df["stops"].astype("int8")
df["weekday"]=df["weekday"].astype("int8")
df["duration"]=df["duration"].astype("float16")
df["dept_hours"]=df["dept_hours"].astype("int8")
df["arrival_hours"]=df["arrival_hours"].astype("int8")
df["Dept_date"]=df["Dept_date"].astype("int8")

In [61]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13139 entries, 0 to 13138
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype          
---  ------                --------------  -----          
 0   Price                 13139 non-null  int32          
 1   Cities                13139 non-null  object         
 2   departure_time        13139 non-null  object         
 3   arrival_time          13139 non-null  object         
 4   layeover              13139 non-null  object         
 5   Airline               13139 non-null  object         
 6   Scraping_Time         13139 non-null  object         
 7   Cabin                 13139 non-null  object         
 8   Dept_city             13139 non-null  object         
 9   Dept_date             13139 non-null  int8           
 10  Dept_month            13139 non-null  object         
 11  arrival_city          13139 non-null  object         
 12  stops                 13139 non-null  int8           
 13  D

In [62]:
df["scraping_hours"]=pd.to_datetime(df["Scraping_Time"]).dt.hour
# df["Departure_Date"]=pd.to_datetime(df["Departure_Date"])
df["optimal_hours"]= df['dept_hours'] - df['scraping_hours']

In [63]:
df.drop(columns=["Departure_Date","Arrival_Date","day_name","arrival_flights_time","arrival_hours",'Dept_month',"Scraping_Time","scraping_hours"],inplace=True,axis=1)

In [64]:
df=df.dropna()

In [65]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13139 entries, 0 to 13138
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype          
---  ------             --------------  -----          
 0   Price              13139 non-null  int32          
 1   Cities             13139 non-null  object         
 2   departure_time     13139 non-null  object         
 3   arrival_time       13139 non-null  object         
 4   layeover           13139 non-null  object         
 5   Airline            13139 non-null  object         
 6   Cabin              13139 non-null  object         
 7   Dept_city          13139 non-null  object         
 8   Dept_date          13139 non-null  int8           
 9   arrival_city       13139 non-null  object         
 10  stops              13139 non-null  int8           
 11  duration_time      13139 non-null  timedelta64[ns]
 12  duration           13139 non-null  float16        
 13  weekday            13139 non-null  int8       

In [66]:
df.drop(columns=['layeover'],inplace=True)

In [67]:
df.head()

,Price,Cities,departure_time,arrival_time,Airline,Cabin,Dept_city,Dept_date,arrival_city,stops,duration_time,duration,weekday,dept_hours,Dept_flights_time,optimal_hours
0,6855,"New Delhi to Chennai , 10 May",18:55,21:35,Spicejet,PE,New Delhi,10,Chennai,0,0 days 02:40:00,160.0,0,18,Evening_flight,2
1,12168,"New Delhi to Chennai , 10 May",19:10,13:15,Spicejet,PE,New Delhi,10,Chennai,1,-1 days +18:05:00,1085.0,0,19,Evening_flight,3
2,10096,"New Delhi to Guwahati , 10 May",19:10,20:25,Spicejet,PE,New Delhi,10,Guwahati,1,0 days 01:15:00,75.0,0,19,Evening_flight,3
3,11788,"New Delhi to Patna , 10 May",19:10,20:50,Spicejet,PE,New Delhi,10,Patna,0,0 days 01:40:00,100.0,0,19,Evening_flight,3
4,6147,"Mumbai to New Delhi , 10 May",19:45,21:55,Vistara,PE,Mumbai,10,New Delhi,0,0 days 02:10:00,130.0,0,19,Evening_flight,3


In [68]:
for i in range(df.shape[0]):
    if df['optimal_hours'].iloc[i]<0:
    
        df['optimal_hours'].iloc[i] = 24 + df['optimal_hours'].iloc[i]
    # print(df['optimal_hours'].iloc[i])

C:\Users\Dell\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [69]:
df.optimal_hours.describe()

count    13139.000000
mean        12.397138
std          7.000466
min          0.000000
25%          6.000000
50%         13.000000
75%         19.000000
max         23.000000
Name: optimal_hours, dtype: float64

In [70]:
df.optimal_hours.nunique()

24

In [71]:
air=df["Airline"].value_counts().sort_values(ascending=False)
air
l1 = []
l1.append(df[df["Airline"] == "flybig"].index)
l1.append(df[df["Airline"] == "TruJet"].index)
l1.append(df[df["Airline"] == "Air India Express"].index)
#print(l1)
l2 = []
for i in l1:
    for j in i:
        l2.append(j)
df.drop(l2,inplace=True,axis=0)

In [72]:
df.dropna(inplace=True,how='any')

In [73]:
df.to_csv("trainable_pe.csv",index=False)

In [44]:
df1 = pd.read_csv("trainable_pe.csv")
df1.head()

,Price,Cities,departure_time,arrival_time,Airline,Cabin,Dept_city,Dept_date,arrival_city,stops,duration_time,duration,weekday,dept_hours,Dept_flights_time,optimal_hours
0,6855,"New Delhi to Chennai , 10 May",18:55,21:35,Spicejet,PE,New Delhi,10,Chennai,0,0 days 02:40:00,160.0,0,18,Evening_flight,2
1,12168,"New Delhi to Chennai , 10 May",19:10,13:15,Spicejet,PE,New Delhi,10,Chennai,1,-1 days +18:05:00,1085.0,0,19,Evening_flight,3
2,10096,"New Delhi to Guwahati , 10 May",19:10,20:25,Spicejet,PE,New Delhi,10,Guwahati,1,0 days 01:15:00,75.0,0,19,Evening_flight,3
3,11788,"New Delhi to Patna , 10 May",19:10,20:50,Spicejet,PE,New Delhi,10,Patna,0,0 days 01:40:00,100.0,0,19,Evening_flight,3
4,6147,"Mumbai to New Delhi , 10 May",19:45,21:55,Vistara,PE,Mumbai,10,New Delhi,0,0 days 02:10:00,130.0,0,19,Evening_flight,3


In [45]:
df1.shape

(12129, 16)

In [25]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264882 entries, 0 to 264881
Data columns (total 23 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Price                 264882 non-null  int64  
 1   Cities                264882 non-null  object 
 2   departure_time        264882 non-null  object 
 3   arrival_time          264882 non-null  object 
 4   layeover              264882 non-null  object 
 5   Airline               264882 non-null  object 
 6   Scraping_Time         264882 non-null  object 
 7   Cabin                 264882 non-null  object 
 8   Dept_city             264882 non-null  object 
 9   Dept_date             264882 non-null  int64  
 10  Dept_month            264882 non-null  object 
 11  arrival_city          264882 non-null  object 
 12  stops                 264882 non-null  int64  
 13  Departure_Date        264882 non-null  object 
 14  Arrival_Date          264882 non-null  object 
 15  

In [130]:
df2 = pd.read_csv("ready_dataset.csv", sep=',')
df2.head()

,Price,Airline,Cabin,Dept_city,Dept_date,Dept_month,arrival_city,stops,layover_cities,Departure_Date,Arrival_Date,duration,weekday,day_name,dept_hours,arrival_hours,Dept_flights_time,arrival_flights_time
0,6159,Vistara,PE,New Delhi,20,Apr,Mumbai,0,No Stops,20-04-2021 09:30,20-04-2021 11:40,130,1,Tuesday,9,11,Morning_flight,Afternoon_flight
1,6159,Vistara,PE,New Delhi,20,Apr,Mumbai,0,No Stops,20-04-2021 15:30,20-04-2021 17:40,130,1,Tuesday,15,17,Afternoon_flight,Evening_flight
2,6159,Vistara,PE,New Delhi,20,Apr,Mumbai,0,No Stops,20-04-2021 16:55,20-04-2021 19:05,130,1,Tuesday,16,19,Evening_flight,Evening_flight
3,6159,Vistara,PE,New Delhi,20,Apr,Mumbai,0,No Stops,20-04-2021 12:45,20-04-2021 15:00,135,1,Tuesday,12,15,Afternoon_flight,Afternoon_flight
4,6159,Vistara,PE,New Delhi,20,Apr,Mumbai,0,No Stops,20-04-2021 17:45,20-04-2021 20:05,140,1,Tuesday,17,20,Evening_flight,Evening_flight


In [131]:
df2 = df2.append(df1)

In [132]:
df2.to_csv("final_dataset.csv",index=False)

In [133]:
df3 = pd.read_csv("final_dataset.csv")

In [134]:
df3.shape

(325484, 18)

In [135]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 325484 entries, 0 to 325483
Data columns (total 18 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Price                 325484 non-null  int64  
 1   Airline               325484 non-null  object 
 2   Cabin                 325484 non-null  object 
 3   Dept_city             325484 non-null  object 
 4   Dept_date             325484 non-null  int64  
 5   Dept_month            325484 non-null  object 
 6   arrival_city          325484 non-null  object 
 7   stops                 325484 non-null  int64  
 8   layover_cities        325484 non-null  object 
 9   Departure_Date        325484 non-null  object 
 10  Arrival_Date          325484 non-null  object 
 11  duration              325484 non-null  float64
 12  weekday               325484 non-null  int64  
 13  day_name              325484 non-null  object 
 14  dept_hours            325484 non-null  int64  
 15  

In [136]:
df3.columns

Index(['Price', 'Airline', 'Cabin', 'Dept_city', 'Dept_date', 'Dept_month',
       'arrival_city', 'stops', 'layover_cities', 'Departure_Date',
       'Arrival_Date', 'duration', 'weekday', 'day_name', 'dept_hours',
       'arrival_hours', 'Dept_flights_time', 'arrival_flights_time'],
      dtype='object')

In [137]:
df2.columns

Index(['Price', 'Airline', 'Cabin', 'Dept_city', 'Dept_date', 'Dept_month',
       'arrival_city', 'stops', 'layover_cities', 'Departure_Date',
       'Arrival_Date', 'duration', 'weekday', 'day_name', 'dept_hours',
       'arrival_hours', 'Dept_flights_time', 'arrival_flights_time'],
      dtype='object')

In [138]:
df3.tail()

,Price,Airline,Cabin,Dept_city,Dept_date,Dept_month,arrival_city,stops,layover_cities,Departure_Date,Arrival_Date,duration,weekday,day_name,dept_hours,arrival_hours,Dept_flights_time,arrival_flights_time
325479,9114,Vistara,PE,Raipur,16,May,Srinagar,1,New Delhi,2021-05-16 08:55:00,2021-05-16 13:10:00,255.0,6,Sunday,8,13,Morning_flight,Afternoon_flight
325480,9674,Vistara,PE,Raipur,16,May,Ranchi,1,New Delhi,2021-05-16 08:55:00,2021-05-16 19:40:00,645.0,6,Sunday,8,19,Morning_flight,Evening_flight
325481,9674,Vistara,PE,Raipur,16,May,Ranchi,1,New Delhi,2021-05-16 20:20:00,2021-05-16 08:20:00,720.0,6,Sunday,20,8,Evening_flight,Morning_flight
325482,14602,Vistara,PE,Raipur,16,May,Port Blair,2,"New Delhi,Kolkata",2021-05-16 20:20:00,2021-05-16 11:20:00,900.0,6,Sunday,20,11,Evening_flight,Afternoon_flight
325483,15554,Vistara,PE,Raipur,16,May,Port Blair,2,"New Delhi,Kolkata",2021-05-16 08:55:00,2021-05-16 11:20:00,145.0,6,Sunday,8,11,Morning_flight,Afternoon_flight


In [139]:
df3.shape

(325484, 18)

In [140]:
df3.drop_duplicates(inplace=True)

In [141]:
df3.to_csv("final_dataset.csv",index=False)

In [142]:
df4 = pd.read_csv("final_dataset.csv")

In [143]:
df4.shape

(315985, 18)

In [145]:
df4.isnull().sum()

Price                   0
Airline                 0
Cabin                   0
Dept_city               0
Dept_date               0
Dept_month              0
arrival_city            0
stops                   0
layover_cities          0
Departure_Date          0
Arrival_Date            0
duration                0
weekday                 0
day_name                0
dept_hours              0
arrival_hours           0
Dept_flights_time       0
arrival_flights_time    0
dtype: int64

In [26]:
df.isnull().sum()

Price                   0
Cities                  0
departure_time          0
arrival_time            0
layeover                0
Airline                 0
Scraping_Time           0
Cabin                   0
Dept_city               0
Dept_date               0
Dept_month              0
arrival_city            0
stops                   0
Departure_Date          0
Arrival_Date            0
duration_time           0
duration                0
weekday                 0
day_name                0
dept_hours              0
arrival_hours           0
Dept_flights_time       0
arrival_flights_time    0
dtype: int64